# Ejercicio EvCont4 (c) - Compresión LZZ7-AUDIO

* Óscar Sementé Solà
* Abdelkarim Azzouguagh Ouniri
* Rodrigo Cabezas Quirós

In [18]:
import math, re

class LZZ7Compressor:
    
    def __init__(self, win, wsld):
        self.win = win
        self.wsld = wsld
        self.win_b = int(math.log2(win))
        self.wsld_b = int(math.log2(wsld))
    
    def __is_power_of_two(self, v):
        return math.log2(v).is_integer()
    
    def __ld_to_bin(self, l, d, useLD):
        """
        Transforma (L, D) a formato binario de longitud fija.
        """
        b_win = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.win_b)]))
        b_wsld = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.wsld_b)]))
        if useLD:
            return (l, d)
        return b_win(l) + b_wsld(d)
    
    def __match_pattern(self, slide, inp, useLD):
        """
        Busca un patrón de la ventana de entrada en la ventana deslizante. Si lo encuentra devuelve
        (L, D) en decimal o en formato binario (según valor de 'useLD') y la longitud del patrón
        encontrado. Si no lo encuentra devuelve False y 1 (incremento del offset).
        """
        occ, seqs = [], [inp[:i] for i in range(len(inp), 0, -1)]
        while seqs:
            current = seqs.pop(0)
            occ = [(x.start(0), x.end(0)) for x in re.finditer(current, slide)]
            if occ:
                si, ei = occ[-1]
                return self.__ld_to_bin(len(slide[si:ei]), len(slide) - si, useLD), ei - si
        return False, 1
    
    def compress(self, m, useLD=False):
        """
        Compresión de una string binaria a partir de la configuración del compresor.
        """
        if not self.win <= self.wsld and not len(m) >= self.win + self.wsld:
            """
            Que los bits de conf. sean potencia de 2 no tiene ningún sentido. El ejercicio evaluado en
            clase tenía por parametros 8 y 6 bits. Aplicando la norma de la potencia de 2, con 6 bits no
            funcionaria el algoritmo. Así que hemos aplicado esta política.
            or \ not self.__is_power_of_two(self.win) \ or not self.__is_power_of_two(self.wsld):
            """
            return -1
        # Se aplica método de inserción de bit, para los casos problematicos.
        offset, m = 0, self.__add_insertion_bits(m)
        ret = [m[:self.wsld]]
        while offset + self.wsld + self.win <= len(m):
            # Búsqueda de patrón de la ventana de entrada en la deslizante.
            slide = m[offset:offset + self.wsld]
            inp = m[offset + self.wsld:offset + self.wsld + self.win]
            found, flen = self.__match_pattern(slide, inp, useLD)
            if found:
                # Si se encuentra se añade al retorno.
                ret.append(found)
            else:
                # Si no se guarda el símbolo (en formato (L, D) tal y como especifica el enunciado).
                ret.append(self.__ld_to_bin(1, 1, useLD))
            offset -= -flen
        # Si al acabar quedan bits fuera de las ventanas, se añaden al final de la cadena.
        if offset < len(m):
            ret.append(m[offset + self.wsld:])
        if useLD:
            return ret
        return "".join(ret)
    
    def uncompress(self, m):
        """
        Descompresión de una string binaria a partir de la configuración del compresor.
        """
        ret = m[:self.wsld]
        div = [m[i:i+self.wsld_b+self.win_b] for i in range(self.wsld, len(m), self.wsld_b + self.win_b)]
        lf = lambda x: 2**self.win_b if x == 0 else x
        df = lambda x: 2**self.wsld_b if x == 0 else x
        while div:
            c = div.pop(0)
            if len(c) < self.wsld_b + self.win_b:
                ret = ret + c
            else:
                l, d = lf(int(c[:self.win_b], 2)), df(int(c[self.win_b:], 2))
                ret = ret + ret[len(ret) - d: len(ret) + l - d]
        return self.__remove_insertion_bits(ret)
    
    def __add_insertion_bits(self, m):
        """
        Aplica método de inserción de bit para las entradas con un solo carácter.
        """
        asd = [m[i:i + self.wsld - 1] for i in range(0, len(m), self.wsld - 1)]
        opposite = lambda x: "0" if x == "1" else "1"
        for i in range(len(asd)):
            if asd[i] == asd[i][0] * len(asd[i]) and len(asd[i]) == self.wsld - 1:
                asd[i] = asd[i] + opposite(asd[i][0])
        return "".join(asd)
    
    def __remove_insertion_bits(self, m):
        """
        Desaplica método de inserción de bit para las entradas con un solo carácter.
        """
        opposite = lambda x: "0" if x == "1" else "1"
        for i in range(0, len(m), self.wsld):
            p = m[i:i + self.wsld]
            if p[:-1] == p[0] * len(p[:-1]) and p[-1] == opposite(p[0]):
                m = m[:i + self.wsld-1] + m[i+self.wsld:]
        return m

In [19]:
def test(msg, comp, i=-1):
    if i > -1: print("Test #{}".format(i))
    print("-> Original:\t\t\t", msg)
    print("-> Compresión:\n\tCon (L, D):\t\t", comp.compress(msg, useLD=True))
    asd = comp.compress(msg)
    print("\tBinario:\t\t", asd)
    asd2 = comp.uncompress(asd)
    print("-> Descompresión:\t\t", asd2)

msg = "11011100101001111010100010001"
comp = LZZ7Compressor(6, 8)
test(msg, comp)

-> Original:			 11011100101001111010100010001
-> Compresión:
	Con (L, D):		 ['11011100', (3, 7), (3, 5), (1, 1), (2, 2), (3, 8), (2, 2), (1, 1), (4, 4), '01']
	Binario:		 11011100111111110101001100101100010010010010010001
-> Descompresión:		 11011100101001111010100010001


#### [Apartado 1] Haced las modificaciones necesarias en el programa anterior para que sea capaz de leer ficheros de audio en formato .wav, obtener el código binario natural de cada unos de los enteros (int) de los que se compone el sonido (16bits por dato) y devolverlo como una String binaria procesable por vuestros métodos de compresión /descompresión en LZ-77.

In [15]:
def DecimalToBinary(num):
     
    if num >= 1:
        DecimalToBinary(num // 2)
    return str(num % 2)

In [21]:
import wave
def readWavFile(file_name):
    w = wave.open(file_name, "rb")
    binary_data = w.readframes(w.getnframes())
    txt = ''
    for i in binary_data:
        txt += DecimalToBinary(i)
    w.close()
    return txt

readWavFile("data.wav")

'110101110101010110001000001000100000100010000010001000001010001000000010000000000010001000000010000000101000100010001001111101011101111101011111000000100011011111111010001000100000000001110010000010000000100000111101011100000000000001111101010111011101011111010100101010101101110111111111000000100010000101110101010111110101011101011010100010001000001000000000001001110101011101010111000010001000000000000000000000101000000001111101111000100011010111110101001000000010100000110101111101010101111000100000001001010111011101110000001000001000001000001101010111011111111101101000001000001011110111010111011101011101111111011111010111011111110101111101111101011101001010101010100010100000000000000010100010000011110101011000100000110101111101010111011111111111000000101000100111110100100000000000111111011111010111111111010101011000000000000101111101111101010101010000000000000000001010000000100000000010100000000000001000000000100010001010001010000000000010101010001101111101010111011111011111010010000

In [20]:
import cv2
import numpy

def readImage(file_name):
    img = cv2.imread(file_name)
    r = img[:,:,0]      #Channel R
    g = img[:,:,1]      #Channel G
    b = img[:,:,2]      #Channel B
    r = r.flatten()     #2D to 1D
    g = g.flatten()
    b = b.flatten()
    rgb = numpy.concatenate([r,g,b])     #Concatenate 3 channels
    rgb_bin = ''
    for i in rgb:                        #Convert to binary with 8 digits
        rgb_bin += format(i, "08b")
    return rgb_bin

readImage("cubo_LZ77.bmp")

'...'